In [1]:
import cv2
import matplotlib.pyplot as plt
import cvlib as cv
from cvlib.object_detection import draw_bbox
import pyautogui
from PIL import ImageOps, Image
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'


In [39]:
cnt_cars = 0
idi = []
idi.append(cnt_cars)
idi[cnt_cars] = 0
cbox = 0 
xcoord = []
ccoox0 = []
ccoox0.append(0)
ccoox1 = []
ccoox1.append(0)
ccoox2 = []
ccoox2.append(0)
ccoox3 = []
ccoox3.append(0)
fstr = []
a=0
img = pyautogui.screenshot(region=(250, 500, 1250, 500))        
while True:
    img2 = pyautogui.screenshot(region=(250, 500, 1250, 500))
    while img != img2:
        img = pyautogui.screenshot(region=(250, 500, 1250, 500))
        img2 = img
        im = np.array(img)
        # bbox, label, conf = cv.detect_common_objects(im)
        obj = cv.detect_common_objects(im)
        for i in range(len(obj[1])):
            if obj[1][i] == 'car' or obj[1][i] == 'truck' or obj[1][i] == 'bus':
                parado = 0
                for j in range(len(ccoox0)):
                    if len(ccoox0) >= len(obj[1]):
                        rm = 1
                    else:
                        rm = 0
                    difx0 = abs(abs(ccoox0[j]) - abs(obj[0][i][0]))
                    difx1 = abs(abs(ccoox1[j]) - abs(obj[0][i][1]))
                    x0 = obj[0][i][0]
                    y0 = obj[0][i][1]
                    x1 = obj[0][i][2]
                    y1 = obj[0][i][3]
                    difx2 = abs(abs(ccoox2[j]) - abs(obj[0][i][2]))
                    difx3 = abs(abs(ccoox3[j]) - abs(obj[0][i][3]))
                    if difx0 < 20 and difx1 < 20 and difx2 < 20 and difx3 < 20:
                    # if difx0 < 40 and difx1 < 40:
                        a=1
                        mit = int(y0 + ((y1 - y0)/2))
                        if mit > 330 and mit < 400:
                            im_crop = im[mit:y1, x0:x1]
                            gray = cv2.cvtColor(im_crop, cv2.COLOR_BGR2GRAY)
                            gray = cv2.blur(gray,(1,1))
                            gray = cv2.bilateralFilter(gray, 8, 17, 17)
                            canny = cv2.Canny(gray,50,120)
                            cnts,_ = cv2.findContours(canny,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
                            for c in cnts:
                                area = cv2.contourArea(c)
                                x,y,w,h = cv2.boundingRect(c)
                                epsilon = 0.018*cv2.arcLength(c,True)
                                approx = cv2.approxPolyDP(c,epsilon,True)
                                if len(approx)==4 and area>900:
                                    aspect_ratio = float(w)/h
                                    if aspect_ratio>3.4 and aspect_ratio<6:
                                        placa = gray[y:y+h,x:x+w]
                                        text = pytesseract.image_to_string(placa,config='--psm 11')
                                        if len(text) >= 4:
                                            fstr.append(text)
                                            print('PLACA: ',text)
                        if ccoox1[j] < 360 and y0 >= 360:
                            cnt_cars += 1               
                if a == 1:
                    ccoox0[i] = obj[0][i][0]
                    ccoox1[i] = obj[0][i][1]
                    ccoox2[i] = obj[0][i][2]
                    ccoox3[i] = obj[0][i][3]
                    a = 0
                else:
                    if rm == 1:
                        ccoox0[i] = obj[0][i][0]
                        ccoox1[i] = obj[0][i][1]
                        ccoox2[i] = obj[0][i][2]
                        ccoox3[i] = obj[0][i][3]
                        a = 0
                    else:
                        ccoox0.append(obj[0][i][0])
                        ccoox1.append(obj[0][i][1])
                        ccoox2.append(obj[0][i][2])
                        ccoox3.append(obj[0][i][3])
                        a = 0
        print(ccoox0, ccoox1, ccoox2, ccoox3)             
        print('Nº de coches: ', cnt_cars)
        print("MATRICULAS: ",fstr)
        output_image = draw_bbox(im, obj[0], obj[1], obj[2])
        recta = cv2.rectangle(im, (0,360), (1250, 360), (255,0,0), 1)
        cv2.imshow('Detection', output_image)
        cv2.imshow('Detection', recta)
  
       
    if cv2.waitKey(1) == ord('q'): # si el usuario presiona q paramos de grabar.

        break
        

cv2.destroyAllWindows() # cerrar la ventana

ValueError: not enough values to unpack (expected 2, got 0)